In [ ]:
import numpy as np
n = 90

theta_samples = np.linspace(0, 10 * np.pi, n)

def polar_to_ortho_3d(r, theta, phi):
    x = r * np.array(np.sin(theta) * np.cos(phi))
    y = r * np.array(np.cos(theta))
    z = r * np.array(np.sin(theta) * np.sin(phi))
    return np.array([x, y, z])

def polar_to_ortho_2d(r, phi):
    x = r * np.array(np.sin(phi))
    y = r * np.array(np.cos(phi))
    return np.array([x, y])

def heart_curve(samples):
    a = 0.5
    r = np.array(a * (1 + np.cos(samples)))
    delta_r = np.array([1.] + list(r[:-1])) - r;
    phi = samples;
    x_y = polar_to_ortho_2d(r, phi)
    z =  -2 * np.ones((samples.shape));

    radius_ratio = np.exp(30 * (np.abs(delta_r) / (np.max(r) - np.min(r))));
    return  np.array([x_y[0], x_y[1], z]), radius_ratio.reshape((1,len(radius_ratio)))

def sphere_curve(samples):
    a = 0.5
    r = a * np.ones(samples.shape)
    delta_r = np.array([1.] + list(r[:-1])) - r;
    phi = 20 * samples
    theta = 0.5 * samples
    x_y_z = polar_to_ortho_3d(r, theta, phi)
    radius_ratio = 0.5 + (np.sin(phi % np.pi) + np.sin(theta)) / 2
    radius_ratio += 1.3
    return  np.array([x_y_z[0]+1.4, x_y_z[1]-0.3, x_y_z[2]-3.8]), radius_ratio.reshape((1,len(radius_ratio)))

def spiral_curve_y(samples, positive=True):
    a = 0.25
    b = 0.18
    r = a * np.ones(samples.shape)
    delta_r = np.array([1.] + list(r[:-1])) - r;

    x = a * np.cos(samples) - 0.6
    z = b * np.sin(samples) - 3.5
    if positive:
        y = 0.12 * (samples / np.pi - 4)
    else:
        y = -0.12 * (samples / np.pi - 4)
    return  np.array([x-0.8, y, z-0.1]), np.ones((1, len(samples)))

def spiral_curve_x(samples, positive=True):
    a = 0.25
    b = 0.18
    r = a * np.ones(samples.shape)
    delta_r = np.array([1.] + list(r[:-1])) - r;

    y = a * np.cos(samples) - 0.72
    z = b * np.sin(samples) - 3.5
    if positive:
        x = 0.12 * (samples / np.pi - 4)
    else:
        x = -0.12 * (samples / np.pi - 4)
    return  np.array([x-0.85, y, z-0.1]), np.ones((1, len(samples)))

def equi_spiral_curve(samples):
    a = 4e-2
    b = 1.8e-1
    r = a * np.exp(b * samples / np.pi)
    phi = 8 * samples / 10
    x_y = polar_to_ortho_2d(r, phi)
    radius_ratio = np.sqrt(np.sqrt(phi / np.pi))
    z = 0.8 * ((phi/np.pi) - 9.5) * np.ones(samples.shape)
    return  np.array([x_y[1], x_y[0]+0.02, z-0.3]), radius_ratio.reshape((1,len(radius_ratio)))

sphere_curve_center, sphere_curve_radius_ratio = sphere_curve(np.linspace(0, 2 * np.pi, 180))
equi_spiral_curve_center, equi_spiral_curve_radius_ratio = equi_spiral_curve(theta_samples)

spiral_curve_center_1, spiral_curve_radius_ratio_1 = spiral_curve_y(theta_samples)
spiral_curve_center_2, spiral_curve_radius_ratio_2 = spiral_curve_y(theta_samples, positive=False)

spiral_curve_center_3, spiral_curve_radius_ratio_3 = spiral_curve_x(theta_samples)
spiral_curve_center_4, spiral_curve_radius_ratio_4 = spiral_curve_x(theta_samples, positive=False)

center = np.concatenate((spiral_curve_center_1, spiral_curve_center_2, spiral_curve_center_3, spiral_curve_center_4, \
sphere_curve_center, equi_spiral_curve_center), axis=1)
radius = 0.03 * np.concatenate((spiral_curve_radius_ratio_1, spiral_curve_radius_ratio_2, spiral_curve_radius_ratio_3, spiral_curve_radius_ratio_4, \
sphere_curve_radius_ratio / 2, equi_spiral_curve_radius_ratio / 3), axis=1)
color_1 = np.array([0.7, 0.95, 0.9]).reshape((3,1)) + np.random.rand(3, n) / 10.
color_2 = np.array([0.95, 0.8, 0.7]).reshape((3,1)) + np.random.rand(3, n) / 10.

color_3 = np.array([0.8, 0.95, 0.95]).reshape((3,1)) + np.random.rand(3, n) / 10.
color_4 = np.array([0.95, 0.95, 0.95]).reshape((3,1)) + np.random.rand(3, n) / 10.

color_5 = np.array([1., 1., 1.]).reshape((3,1)) - (1.5 - sphere_curve_radius_ratio) * np.random.rand(3, 180) / 3.

color_6 = np.array([0.95, 0.95, 0.95]).reshape((3,1)) + (equi_spiral_curve_radius_ratio) * np.random.rand(3, n) / 10.

color = np.concatenate((color_1, color_2, color_3, color_4, color_5, color_6), axis=1)
mtype = np.random.rand(1, 5 * n + 180)
print('std::vector<Sphere>{')
for i in range( 5 * n + 180):
    print(f'{{Vector3{{{center[0,i]:.3f},{center[1,i]:.3f},{center[2,i]:.3f}}}, {radius[0,i]:.3f}, {i}}},')
print('},')
print('std::vector<Material>{')
for i in range(5 * n + 180):
    mt = 'Plastic' if mtype[0, i] > 0.5 else 'Mirror'
    print(f'{{MaterialType::{mt},Vector3{{{color[0,i]:.3f},{color[1,i]:.3f},{color[2,i]:.3f}}}}},')
print('},')
